# 12. 图神经网络

## 准备工作
- 安装`torch_geometric`, 依次在命令行执行:
```bash
> pip install torch-cluster
> pip install torch-spline-conv
> pip install torch-sparse
> pip install torch-scatter
> pip install torch-geometric
```

## 课程目标

* 了解图卷机神经网络的基本概念和原理
* 掌握图卷机神经网络的常见模型和训练方法
* 能够应用图卷机神经网络进行实际问题的解决



## 课程内容

### 第一部分：图卷机神经网络简介

* 什么是图卷机神经网络?

> 图卷机神经网络（`Graph Convolutional Neural Networks，GCN`）是一种用于处理图数据的深度学习模型。图数据是指由节点和边组成的结构化数据，例如社交网络、交通网络、生物网络等。图卷机神经网络通过学习图中节点之间的关系，来对图数据进行学习和推理。

* 图卷机神经网络的优势
  - 能够有效地处理图数据中的结构信息。
  - 能够处理大规模图数据。
  - 能够进行多种图数据分析任务。

* 图卷机神经网络的应用
  - 图像分类
  - 文本分类
  - 推荐系统
  - 社交网络分析
  - 生物信息学



### 第二部分：图卷机神经网络的模型

* 基本图卷机模型

基本图卷机模型是一种卷积神经网络，它将卷积操作应用于图数据。在图卷机模型中，每个节点都被看作是卷积核，输入是来自其邻居节点的信息。

假设图 $G = (V, E)$ 中，$V$ 是节点集合，$E$ 是边集合。对于节点 $v \in V$，其邻居节点集合为 $N(v) = \{u \in V | (u, v) \in E\}$。基本图卷机模型的输出为节点 $v$ 的状态 $h_v$，其计算公式如下：

$$h_v = f \left( \sum_{u \in N(v)} h_u W_{vu} \right)$$

其中，$f$ 是激活函数，$W_{vu}$是节点 $v$ 和其邻居节点 $u$ 之间的权重矩阵。

在基本图卷机模型中，卷积操作是通过邻接矩阵来实现的。邻接矩阵 A 是一个对称矩阵，其中 $A_{ij}$ 表示节点 $i$ 和节点 $j$ 之间是否存在边。因此，节点 $v$ 的状态 $h_v$ 可以表示为：

$$h_v = f \left( A_v h \right)$$

其中，$A_v$ 是节点 $v$ 的邻接矩阵。

In [16]:
import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import TUDataset, Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

In [78]:
def gcn_layer(h, A, W, activation=torch.nn.ReLU()):
    """
    基本图卷机层
    Args:
        h: 节点的特征矩阵，shape=(n_nodes, d_in)
        A: 邻接矩阵，shape=(n_nodes, n_nodes)
        W: 权重矩阵，shape=(d_in, d_out)
        activation: 激活函数

    Returns:
        h_v: 节点的状态矩阵，shape=(n_nodes, d_out)
    """
    # 用邻接矩阵实现节点之间的属性值传递
    h_v = torch.matmul(A, h) @ W
    h_v = activation(h_v)
    return h_v


In [79]:
# 生成节点特征矩阵: 10个节点，每个节点16维特征
h = torch.rand(10, 16, dtype=torch.float)
# 生成邻接矩阵: 10个节点，每个节点10个邻居
A = torch.randint(0, 2, (10, 10), dtype=torch.float)
# 生成权重矩阵: 16维特征转换为32维特征
W = torch.rand(16, 32, dtype=torch.float)

# 计算节点状态: 10个节点，每个节点32维特征
h_v = gcn_layer(h, A, W)
print(h_v.shape)

torch.Size([10, 32])


- `torch_geometric` 中的基本图卷机模型

In [29]:
class GCN(nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)  

- 数据集的组织形式

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)  # 2 edges 0 <-> 1, 1 <-> 2，注意这里的值是x的索引
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)  # 3 nodes.
data = Data(x=x, edge_index=edge_index)

In [72]:
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
device = torch.device("cpu")
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)

In [69]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [68]:
data.x.size()  # 节点特征矩阵: 2708个节点，1433维特征

torch.Size([2708, 1433])

In [55]:
data.edge_index  # 邻接矩阵: 10556条边，每条边由两个节点的索引组成, 注意组织形式： (2, n_edges)

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [65]:
data.y  # 节点标签: 2708个节点, 标签为0~6

tensor([3, 4, 4,  ..., 3, 3, 3])

In [75]:
data.train_mask  # 训练集掩码: 2708个节点, 1表示训练集，0表示非训练集

tensor([ True,  True,  True,  ..., False, False, False])

In [76]:
data.val_mask  # 验证集掩码: 2708个节点, 1表示验证集，0表示非验证集

tensor([False, False, False,  ..., False, False, False])

In [77]:
data.test_mask  # 测试集掩码: 2708个节点, 1表示测试集，0表示非测试集

tensor([False, False, False,  ...,  True,  True,  True])

In [27]:
num_node_features = dataset.num_node_features  # 节点特征维度
num_classes = dataset.num_classes  # 节点标签类别数
print(num_node_features, num_classes)

- 模型

In [73]:
model = GCN(num_node_features, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [74]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7990


* 扩展图卷机模型


* 图卷机模型的选择



### 第三部分：图卷机神经网络的训练

* 图卷机神经网络的损失函数
* 图卷机神经网络的优化算法
* 图卷机神经网络的训练策略

### 第四部分：图卷机神经网络的应用

* 图像分类
* 文本分类
* 推荐系统
* 社交网络分析



## 教学方法

* 讲授：介绍图卷机神经网络的基本概念和原理
* 演示：演示图卷机神经网络的模型和训练方法
* 讨论：讨论图卷机神经网络的应用



## 课程安排

### 第一部分：图卷机神经网络简介（20分钟）

* 介绍图卷机神经网络的基本概念和原理
* 介绍图卷机神经网络的优势
* 介绍图卷机神经网络的应用

### 第二部分：图卷机神经网络的模型（30分钟）

* 介绍基本图卷机模型
* 介绍扩展图卷机模型
* 介绍图卷机模型的选择

### 第三部分：图卷机神经网络的训练（20分钟）

* 介绍图卷机神经网络的损失函数
* 介绍图卷机神经网络的优化算法
* 介绍图卷机神经网络的训练策略

### 第四部分：图卷机神经网络的应用（20分钟）

* 介绍图像分类的应用
* 介绍文本分类的应用
* 介绍推荐系统的应用
* 介绍社交网络分析的应用



## 课程作业

* 阅读相关文献，了解图卷机神经网络的最新进展
* 实现一个图卷机神经网络，并在公开数据集上进行训练和测试

## 课程评价

* 课堂参与
* 课程作业
* 期末考试

## 课程补充

* 图卷机神经网络是近年来人工智能领域的一个重要研究方向，在图像分类、文本分类、推荐系统等领域取得了显著的成果。
* 本课程将介绍图卷机神经网络的基本概念和原理，以及常见的模型和训练方法。
* 希望通过本课程的学习，学生能够掌握图卷机神经网络的基本知识和技能，并能够应用图卷机神经网络进行实际问题的解决。


- 导入`torch_geometric`

- 构建GNN